# Modelado Clásico - Actividad 1

Topic modeling es una técnica de aprendizaje automático no supervisado donde intentados descubrir tópicos que son abstractos al texto pero que pueden describir una colección de documentos. Es importante marcar que estos "tópicos" no son necesariamente equivalentes a la interpretación coloquial de tópicos, sino que responden a un patrón que emerge de las palabras que están en los documentos.

La suposición básica para Topic Modeling es que cada documento está representado por una mescla de tópicos, y cada tópico consite en una conlección de palabras.

## Direcciones
Intentaremos construir un pipeline de machine learning donde como entrada recibamos texto, ejecutemos todos los pasos que vimos en este notebook incluyendo:

 - Eliminación de stopwords
 - Tokenización
 - Stemming y Lemmatization
 - Procesamiento especico del tema
 - Creación de features utilizando algun metodo de reducción de dimensionalidad, SVD, LSI, LDA

, para luego utilizar estas features para entrenar un modelo que nos permita predecir alguna propiedad interesante del set de datos. En este caso en particular, donde estermos analizando tweets, predeciremos el sector al que pertenece el tweet: Alimentación, Bebidas, etc.
 
En esta actividad les proponemos realizar cambios en alguna de las etapas del procesamiento para modificar la performance del modelo resultante y evaluar que cambios generan el mejor modelo resultante.

<img src='https://github.com/santiagxf/M72109/blob/master/NLP/Docs/atap_0406.png?raw=1' />

En particular deberan contestar las siguientes preguntas:

 - ¿Que métodos de reducción de dimensionalidad resultan mejores?
 - ¿Que numero de componentes hace sentido para estre problema?
 - ¿Que tipo de modelos resultan mejores a la hora de ser utilizados como clasificadores? (estimator)

> **Pista:** ¿Como podrían explorar esta multiplicidad de opciones de forma automática?

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/classic/topic-modeling.txt \
    --quiet --no-clobber
!pip install -r topic-modeling.txt --quiet

In [ ]:
!python -m spacy download es_core_news_sm

Primero importaremos algunas librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### Sobre el set de datos con el que vamos a trabajar

Utilizaremos como ejemplo un set de datos en español que contiene tweets que diferentes usuarios han publicado en relación a diferentes marcas de productos u empresas en el rubro de alimentación, construcción, automoviles, etc. Estos tweets, a su vez, están asociados a una de las diferentes fases en el proceso de ventas (también conocido como Marketing Funel) y por eso están tagueados con las fases de:
 - Awareness – el cliente es conciente de la existencia de un producto o servicio
 - Interest – activamente expresa el interes de un producto o servicio
 - Evaluation – aspira una marca o producto en particular
 - Purchase – toma el siguiente paso necesario para comprar el producto o servicio
 - Postpurchase - realización del proceso de compra. El cliente compara la diferencia entre lo que deseaba y lo que obtuvo

Referencia: [Spanish Corpus of Tweets for Marketing](http://ceur-ws.org/Vol-2111/paper1.pdf

> Nota: La version de este conjunto de datos que utilizaremos aqui es una versión preprocesada del original.

In [ ]:
tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

## Desarrollo

### Creando nuestros sets de datos de entrenamiento y testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

### Construcción del modelo: Pasos

**Paso 1:** Instanciamos nuestro preprocesamiento de texto

In [ ]:
import unidecode
import spacy
import es_core_news_sm as spa
import re
import sklearn
import nltk
from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

class TextNormalizer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self):
        nltk.download('stopwords', quiet=True)

        self.parser = spa.load() # Cargamos el parser en español
        self.tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # Creamos un tokenizer
        self.stemmer = stem.SnowballStemmer(language='spanish') # Creamos un steammer
        self.lemmatizer = lambda word : " ".join([token.lemma_ for token in self.parser(word)]) # Creamos un lemmatizer
        self.stopwords = set(stopwords.words('spanish')) # Instanciamos las stopwords en español
        self.urls_regex = re.compile('http\S+') # Usamos una expresion regular para encontrar las URLs
    
    def process_text(self, text):
        tokens = self.tokenizer.tokenize(text)
        tokens = [token for token in tokens if not re.match(self.urls_regex, token)]
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in self.stopwords]
        tokens = [unidecode.unidecode(token) for token in tokens] # Quitamos acentos
        tokens = [self.lemmatizer(token) for token in tokens]
        return tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for doc in X:
            yield ' '.join(self.process_text(text=doc))

In [ ]:
normalizer = TextNormalizer()

**Paso 2:** Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

**Paso 3:** Instanciamos nuestro generador de features

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

# featurizer = 

**Paso 4:** Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# estimator =

### Pipeline

Ensamblamos el pipeline

In [ ]:
from sklearn.pipeline import Pipeline

# pipeline = Pipeline(steps=[('normalizer', normalizer), 
#                           ('vectorizer', vectorizer),
#                           ('featurizer', featurizer),
#                           ('estimator', estimator)])

### Evaluación

**Evaluación:** Entrenamos el modelo y testeamos su performance

In [ ]:
# model = pipeline.fit(X=X_train, y=y_train)

In [ ]:
# predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

# print(classification_report(y_test, predictions))